In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/sportsQnA.csv", dtype="string").dropna()
asian_countries = ["China", "Japan", "South Korea", "Taiwan", "Hong Kong"]

df_asia = df[df['nationality'].isin(asian_countries)] # players from asia
df_usa = df[df['nationality'] == "United States"] # players from america

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

q_asia = df_asia['question']
q_usa = df_usa['question']

def vectorize(data):
  vectorizer = CountVectorizer(stop_words='english')
  X = vectorizer.fit_transform(data).todense()
  output = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
  return output

v_asia = vectorize(q_asia)
v_usa = vectorize(q_usa)